In [9]:
import os
import csv
import cv2

import importlib
import lib.export
import lib.A_acc
import lib.B_eclahe
import lib.C_msds
import lib.show_new
import lib.validation
importlib.reload(lib.export)
importlib.reload(lib.A_acc)
importlib.reload(lib.B_eclahe)
importlib.reload(lib.C_msds)
importlib.reload(lib.show_new)
importlib.reload(lib.validation)
from lib.export import ekspor_png
from lib.A_acc import get_color_corrected_increased_dynamic_range
from lib.B_eclahe import apply_clahe_modified
from lib.C_msds import enhance_L_with_D_lab
from lib.show_new import show_image_group_auto
from lib.validation import getScore_flexible


def batch_lab_metrics(
    input_folder: str,
    output_csv: str,
    cie: bool = True,
    device: str = "cuda",
    exts=(".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"),
    recursive: bool = False,
    start: int = 0,
    limit: int = None
):
    """
    Batch processing: hitung m_a, m_b, M, D_a, D_b, D, cast untuk semua gambar di folder.
    Bisa diproses sebagian (misalnya 100 pertama dulu).

    Params
    ------
    input_folder : str
        Folder berisi gambar.
    output_csv   : str
        Path file CSV output.
    cie          : bool
        Gunakan skala CIE Lab [-128,127] bila True.
    device       : str
        "cuda" untuk GPU, "cpu" untuk CPU.
    exts         : tuple[str]
        Ekstensi file gambar yang diproses.
    recursive    : bool
        Telusuri subfolder jika True.
    start        : int
        Indeks awal gambar yang diproses.
    limit        : int or None
        Jumlah gambar yang diproses. Jika None → semua dari `start` sampai akhir.
    """
    # kumpulkan semua path gambar
    all_files = []
    walker = os.walk(input_folder) if recursive else [(input_folder, [], os.listdir(input_folder))]

    for root, _, files in walker:
        for fname in sorted(files):
            if fname.lower().endswith(exts):
                all_files.append(os.path.join(root, fname))

    total = len(all_files)
    end = total if limit is None else min(start + limit, total)

    rows = []
    iter = 0
    for fpath in all_files[start:end]:
        filename = os.path.basename(fpath)
        # filename, raw_uiqm, raw_uciqe = getScore_flexible(fpath, "")

        im_increased, cast = get_color_corrected_increased_dynamic_range(fpath, device="cuda")
        filename, a_uiqm, a_uciqe = getScore_flexible(im_increased, filename)

        im_eclahed = apply_clahe_modified(im_increased, mode="lab", tile_grid_size=(8,8), mu=0.3, q=1e-6)
        filename, b_uiqm, b_uciqe = getScore_flexible(im_eclahed, filename)

        im_detailed = enhance_L_with_D_lab(im_increased)
        filename, c_uiqm, c_uciqe = getScore_flexible(im_detailed, filename)

        rows.append([
            iter,
            filename,
            cast,
            a_uiqm,
            a_uciqe,
            b_uiqm,
            b_uciqe,
            c_uiqm,
            c_uciqe,
            # metrics["m_a"], metrics["m_b"], metrics["M"],
            # metrics["D_a"], metrics["D_b"], metrics["D"],
            # metrics["cast"]
        ])

        iter = iter + 1

    # tulis CSV (append jika sudah ada)
    write_header = not os.path.exists(output_csv)
    os.makedirs(os.path.dirname(output_csv) or ".", exist_ok=True)
    with open(output_csv, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        if write_header:
            writer.writerow(["id","filename", "cast", "a_uiqm", "a_uciqe", "b_uiqm", "b_uciqe", "c_uiqm", "c_uciqe"])
        writer.writerows(rows)

    print(f"Selesai. Diproses {len(rows)} gambar (index {start} sampai {end-1}) dari total {total}.")


# Run 1: proses 100 pertama 4m 46.9s
batch_lab_metrics("../UIEB/raw-890", "hasil/metrics.csv", start=0, limit=10)

# # Run 2: lanjut 100 berikutnya
# batch_lab_metrics("data/uieb", "hasil/metrics.csv", start=100, limit=100)

# # Run 3: lanjut lagi
# batch_lab_metrics("data/uieb", "hasil/metrics.csv", start=200, limit=100)

Selesai. Diproses 10 gambar (index 0 sampai 9) dari total 890.
